# build the UX experience for the song recommender

- store the list in a .csv
- get input from user
    - check for spelling
    - if song is found in the hot list
        - prompt user to confirm by printing song and artist + requesting confirmation
        - if not, send to alternative recommender
            - make dummy function for this

In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from fuzzywuzzy import process
import difflib

In [5]:
def load_csv():
    # load list of hot songs as possible choices
    df = pd.read_csv("hot_100.csv")
    df = df.apply(lambda x: x.str.lower())
    return df

In [6]:
# load list of hot songs as possible choices
hot_100 = load_csv()

In [64]:
def str_matcher(user_input, song_list):
    # Find the closest matching song title to the input string
    best_match = difflib.get_close_matches(user_input, song_list["song"], n=1, cutoff=0.8)
    # Print the closest matching song title and the corresponding similarity score
    if best_match:
       similarity = difflib.SequenceMatcher(None, user_input, best_match[0]).ratio()
       corrected_input = best_match[0]
       match = True
    else:
        corrected_input = None
        match = False
    return match, corrected_input

In [13]:
str_matcher("flower", hot_100)

['flowers']


In [52]:
def user_input(song_list):
    # get user input
    user_choice = input("Please choose a hot song you like. To exit, type '/exit'.")
    # check if the program should be terminated
    if user_choice == "/exit":
        print("Program terminated.")
    else:
        # return list with [match=True/False, corrected_input]
        user_choice = str_matcher(user_choice, song_list)[1]

        # check if the corrected string appeares in full in the list of songs
        if song_list["song"].str.contains(fr'\b{user_choice}\b').any():
           print("Your song is in the list of 100 hottest songs.")
        else:
            print("Your song is not in the list of the hottest 100 songs, please enter a song from the list:", np.array(song_list["song"].str.title()))
            user_choice = user_input(song_list)
    return user_choice

In [56]:
def good_choice(song_list):
    good_choice = False
    # fish out stinking fish
    while not good_choice:
        # get user input
        user_choice = user_input(song_list)

        # get artist corresponding to song choice
        artist = song_list.loc[song_list["song"].str.lower().str.contains(user_choice),:]["artist"][0]
        # have user confirm choice
        if str_matcher(user_choice, song_list)[1]:
            confirm = input(f"Is this the song you meant? {user_choice.title()} by {artist.title()} \n Type 'y' to confirm and 'n' to deny. To exit, type '/exit'.")
            if confirm != "y":
                good_choice = False
            else:
                good_choice = True
        else:
            good_choice = False

    return user_choice

In [45]:
artist = hot_100.loc[hot_100["song"].str.lower().str.contains("flowers"),:]["artist"][0]

In [65]:
good_choice(hot_100)

Your song is in the list of 100 hottest songs.


'flowers'

In [70]:
def recommender(song_list):
    # call user_input
    user_choice = good_choice(song_list)

    # get the dataframe without the input song
    song_list_out = pd.DataFrame(song_list[~song_list["song"].str.contains(user_choice)])

    # choose random song from the list of songs
    recommended_song = song_list_out.iloc[randint(0, len(song_list)), 0].title()
    print("Based on the input song, we recommend you this song:")

    return recommended_song

In [71]:
recommender(hot_100)

Your song is in the list of 100 hottest songs.
Based on the input song, we recommend you this song:


'Everything I Love'

In [11]:
# not working yet
def check_duplicates(song_list):
    song_list.loc[len(song_list)+1, "song"] = "Flowers"
    song_list.loc[len(song_list)+1, "artist"] = "Blabla"

    user_choice = user_input(song_list)

    song_list.loc[song_list["song"].str.contains(user_choice)]


In [12]:
check_duplicates()

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [ ]:
hot_100 = load_csv()
hot_100.iloc[randint(0, len(hot_100)),0]

In [ ]:

    if hot_100["song"].str.lower().str.contains(user_choice.lower()).sum() > 1:
        print("There are two songs with this name, please input which artist you meant. These are the artists you can choose from:", hot_100.loc[hot_100["song"].str.contains(user_choice)]["artist"])
        choose_artist = input("Type '/exit' to exit the song recommender.")

        #hot_100_out = pd.DataFrame(hot_100[~hot_100["song"].str.contains(user_choice)])
    else:
        print("fatal error")